<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Proyecto-03---Sistemas-de-Recomendación" data-toc-modified-id="Proyecto-03---Sistemas-de-Recomendación-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Proyecto 03 - Sistemas de Recomendación</a></span><ul class="toc-item"><li><span><a href="#Dataset:-STEAM" data-toc-modified-id="Dataset:-STEAM-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dataset: STEAM</a></span><ul class="toc-item"><li><span><a href="#Exploración-de-datos" data-toc-modified-id="Exploración-de-datos-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Exploración de datos</a></span></li><li><span><a href="#Filtro-Colaborativo" data-toc-modified-id="Filtro-Colaborativo-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Filtro Colaborativo</a></span></li><li><span><a href="#Para-pensar,-investigar-y,-opcionalmente,-implementar" data-toc-modified-id="Para-pensar,-investigar-y,-opcionalmente,-implementar-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Para pensar, investigar y, opcionalmente, implementar</a></span></li><li><span><a href="#¡Tómate-tiempo-para-investigar-y-leer-mucho!" data-toc-modified-id="¡Tómate-tiempo-para-investigar-y-leer-mucho!-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span><strong>¡Tómate tiempo para investigar y leer mucho!</strong></a></span></li></ul></li></ul></li></ul></div>

# Proyecto 03 - Sistemas de Recomendación

## Dataset: STEAM

**Recuerda descargar el dataset de [aquí](https://github.com/kang205/SASRec). Son dos archivos, uno de calificaciones y otro de información sobre los juegos.**

En este notebook te dejamos unas celdas para que puedas comenzar a trabajar con este dataset. Sin embargo, **deberás** modificarlas para hacer un mejor manejo de datos. Algunas cosas a las que deberás prestar atención (tal vez no a todas):
1. Tipos de datos: elige tipos de datos apropiados para cada columna.
2. Descartar columnas poco informativas.
3. Guardar en memoria datasets preprocesados para no tener que repetir código que tarde en correr.

### Exploración de datos

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿cuáles son los juegos más populares? ¿Y los menos populares?

### Filtro Colaborativo

Deberás implementar un sistema de recomendación colaborativo para este dataset. Ten en cuenta:

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.

Puedes implementar un filtro colaborativo a partir de la similitud coseno o índice de Jaccard. ¿Puedes utilizar los métodos de la librería Surprise? Si no es así, busca implementaciones (por ejemplo, nuevas librerías) que sean apropiadas.

Para comenzar a trabajar, puedes asumir que cada entrada es un enlace entre una persona usuaria y un item, **independientemente** de si la crítica es buena o mala. 

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Cómo harías para ponerle un valor a la calificación?
1. ¿Cómo harías para agregar contenido? Por ejemplo, cuentas con el género, precio, fecha de lanzamiento y más información de los juegos.
1. ¿Hay algo que te gustaría investigar o probar?

### **¡Tómate tiempo para investigar y leer mucho!**

In [ ]:
import gc
import gzip
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

def parse(path):
    g = gzip.open(path, "r")
    for l in g:
        yield eval(l)

**Reviews**

In [ ]:
contador = 0
data_reviews = []
# Vamos a guardar una de cada 10 reviews para no llenar la memoria RAM. Si pones n=3, 
# abrira uno de cada tres, y asi.
n = 10
for l in parse(".\Proyectos\Proyecto III\steam_reviews.json.gz"):
    if contador%n == 0:
        data_reviews.append(l)
    else:
        pass
    contador += 1

Cargamos el dataset y elegimos algunas columnas a utilizar.

In [ ]:
#data_reviews = pd.DataFrame(data_reviews, columns=["username", "product_id", "early_access",
#                                                   "hours", "date"])
data_reviews = pd.DataFrame(data_reviews)
reviews = data_reviews.copy()
reviews.head()

In [ ]:
reviews.isna().sum()

Buscamos duplicados

In [ ]:
mask = reviews.duplicated()
print("DUPLICADOS")
reviews[mask]

In [ ]:
name = "Spicy Michael"
reviews.query("username ==  @name")

Eliminamos los duplicados y vemos cantidad de usuarios y de juegos a analizados.

In [ ]:
reviews.drop_duplicates(inplace=True)
print("Total de juegos:", reviews.product_id.value_counts().count())
print("Total de usuarios:", reviews.username.value_counts().count())

In [ ]:
reviews.isna().sum()

Graficamos proporción de juegos con acceso anticipado.

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x=data_reviews["early_access"])
plt.title("ACCESO ANTICIPADO", fontsize=15, fontweight="bold")
plt.tick_params(labelsize=15)
plt.xlabel("Early acces".upper(), labelpad=15)
plt.ylabel("Cantidad".upper(), labelpad=15)
plt.show()

Establecemos algunas consultas para entender un poco más el dataset. agrupamos usuarios y calculamos las horas destinadas a jugar.

In [ ]:
# Cantidad de usuarios sin horas cargadas
horas_null = len(reviews.query("hours.isnull()", engine="python"))
horas_cero = len(reviews.query("hours == 0", engine="python"))
print(f"Hay { horas_cero } de usuarios sin horas cargadas y { horas_null } de usuarios sin registro en horas.")

In [ ]:
reviews.groupby("username").agg(total_juegos=pd.NamedAgg(column="product_id", aggfunc="count"),
                                total_horas=pd.NamedAgg(column="hours", aggfunc="sum"),
                                total_comentarios=pd.NamedAgg(column="text", aggfunc="count"))\
                                .sort_values("total_comentarios", ascending=False)

In [ ]:
reviews.groupby("product_id").agg(total_usuarios=pd.NamedAgg(column="username", aggfunc="count"),
                                  total_comentarios=pd.NamedAgg(column="text", aggfunc="count"))\
                                  .sort_values("total_usuarios", ascending=False)

In [ ]:
reviews.groupby("product_id").agg(total_usuarios=pd.NamedAgg(column="username", aggfunc="count"),
                                  total_horas=pd.NamedAgg(column="hours", aggfunc="sum"),
                                  total_comentarios=pd.NamedAgg(column="text", aggfunc="count"))\
                                  .sort_values("total_usuarios", ascending=False)

**Games**

In [ ]:
data_games = []
for games in parse(".\Proyectos\Proyecto III\steam_games.json.gz"):
    data_games.append(games)
#data_games = pd.DataFrame(data_games, columns=[ "id", "app_name", "tags", "specs", "early_access", "sentiment", "metascore", "price" ])

Cargamos el dataset e imprimimos su ```head```.

In [ ]:
data_games = pd.DataFrame(data_games)
games = data_games.copy()
games.head()

Eliminamos las columnas innecesarias y buscamos los valores nulos.

In [ ]:
games.drop(columns=["publisher", "genres", "url", "reviews_url", "sentiment", "metascore"], inplace=True)

**Duplicados**

In [ ]:
duplicados = len(games[games.drop(columns=["tags", "specs"]).duplicated()])
print(f"Total de registros duplicados: { duplicados }")
id = "612880"
games.query("id == @id")

In [ ]:
games.drop([14573], axis=0, inplace=True)

Es importante remarcar que el ```id``` es la columna que no debemos dejar nula, junto a ```app_name```. Estas son las que nos van a vincular los dos dataset.

In [ ]:
games.isna().sum()

Buscamos los nulos de **id** y **app_name**

In [ ]:
games.query('app_name.isna() | id.isna()', engine='python')

In [ ]:
games.groupby("developer").agg(juegos_publicados=pd.NamedAgg(column="early_access", aggfunc="count")).sort_values("juegos_publicados", ascending=False)
